# Load

## Import

In [ ]:
%reload_ext autoreload
%autoreload 2
%reload_ext cython
%reload_ext line_profiler
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import dill
import scipy as sp
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
from ipywidgets import interact, widgets, Layout
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

## Networks

In [ ]:
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
          'energy', 'optics', 'earth science', 'geology', 'meteorology',
          'philosophy of language', 'philosophy of law', 'philosophy of mind',
          'philosophy of science', 'economics', 'accounting', 'education',
          'linguistics', 'law', 'psychology', 'sociology', 'electronics',
          'software engineering', 'robotics',
          'calculus', 'geometry', 'abstract algebra',
          'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra',
          'number theory', 'dynamical systems and differential equations']

In [ ]:
import wiki

path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated'

networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net(path_graph=os.path.join(path_saved, topic + '.pickle'),
                               path_barcodes=os.path.join(path_saved, topic + '.barcode'))

## Models

In [ ]:
base_dir = os.path.join('/', 'Users', 'harangju', 'Developer', 'data', 'wiki', 'simulations')
session_dir = os.path.join(base_dir, '20200312_2216')

In [ ]:
filenames = sorted(os.listdir(session_dir))
filenames[:3]

In [ ]:
filenames[0].split('_')[1]

In [ ]:
# models = {filename.split('_')[1]: dill.load(open(os.path.join(session_dir, filename), 'rb'))
#           for filename in filenames}
model_paths = {filename.split('_')[1]: os.path.join(session_dir, filename) for filename in filenames}

# Analysis

## Basic network statistics

## Persistent homology

# Plot

In [ ]:
import plotly.subplots as sb

In [ ]:
import plotly.express as px
fig = px.colors.qualitative.swatches()
# fig.show()

In [ ]:
first_n_nodes = 10

In [ ]:
for topic in ['anatomy']:#, 'biochemistry', 'cognitive science', 'evolutionary biology',
#               'linguistics', 'law', 'psychology', 'sociology', 'electronics',
#               'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra']:
    fig = sb.make_subplots(1,2)
    network = networks[topic]
    model = dill.load(open(model_paths[topic], 'rb'))
    fig.add_trace(go.Histogram(x=[d for _,d in network.graph.degree], nbinsx=30, 
                               name='empirical'),
                  row=1, col=1)
    fig.add_trace(go.Histogram(x=[d for _,d in model.graph.degree], nbinsx=30, 
                               name='model'),
                  row=1, col=1)
    fig.update_yaxes(title_text='number of edges', row=1, col=1)
    fig.update_xaxes(title_text='degree', row=1, col=1)
    fig.add_trace(go.Scatter(x=sorted([network.graph.nodes[n]['year'] 
                                       for n in network.graph.nodes]),
                             y=list(range(1,len(network.graph.nodes)+1)),
                             mode='lines', name='empirical', showlegend=False,
                             line={'color': px.colors.qualitative.Plotly[0]}),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=sorted([model.graph.nodes[n]['year'] for n in model.graph.nodes]),
                             y=list(range(1,len(model.graph.nodes)+1)),
                             mode='lines', name='model', showlegend=False,
                             line={'color': px.colors.qualitative.Plotly[1]}),
                  row=1, col=2)
    start_date = sorted(list(nx.get_node_attributes(model.graph_parent, 'year').values()))[first_n_nodes]
    fig.add_trace(go.Scatter(x=[start_date, start_date], y=[0, max(len(model.graph.nodes),
                                                                   len(network.graph.nodes))],
                             mode='lines', showlegend=False,
                             line={'color': px.colors.qualitative.Plotly[2],
                                   'dash': 'dash'}),
                  row=1, col=2)
    fig.update_yaxes(title_text='number of nodes', row=1, col=2)
    fig.update_xaxes(title_text='year', row=1, col=2)
    fig.update_layout(title=topic, template='plotly_white')
    fig.show()

In [ ]:
model._cliques = None
model._filtration = None
model._persistence = None
model._barcodes = None
model._numbered = None
model._nodes = []
model._years = []
model._nodes_for_year = {}

In [ ]:
model.barcodes